# BRSSD Traffic Sign Recognition with YOLOv10
## Bangladeshi Road Sign Symbol Dataset

This notebook trains YOLOv10 models on the BRSSD dataset for detecting and recognizing Bangladeshi traffic signs.

## 1. Setup and Installation

In [ ]:
# Install required packages
!pip install -q ultralytics roboflow kaggle opencv-python pillow matplotlib seaborn pyyaml

import os
import yaml
from pathlib import Path
import cv2
import matplotlib.pyplot as plt
from ultralytics import YOLO
import shutil

print("✓ Packages installed")

## 2. Download BRSSD Dataset

In [ ]:
# Option 1: Download from Roboflow
from roboflow import Roboflow

# Enter your Roboflow API key
API_KEY = "YOUR_ROBOFLOW_API_KEY"  # Get from https://app.roboflow.com/settings/api

rf = Roboflow(api_key=API_KEY)
# Update workspace and project name based on actual BRSSD location
project = rf.workspace("YOUR_WORKSPACE").project("brssd")
dataset = project.version(1).download("yolov8", location="./BRSSD")

print("✓ Dataset downloaded")

In [ ]:
# Option 2: Use pre-downloaded dataset or run download script
!python3 download_brssd.py --source all

In [ ]:
# Option 3: Manual setup - if you've already downloaded BRSSD
# Ensure your dataset is structured as:
# BRSSD/
#   train/images/
#   train/labels/
#   val/images/
#   val/labels/
#   test/images/  (optional)
#   test/labels/  (optional)

## 3. Verify Dataset

In [ ]:
# Check dataset structure
dataset_path = Path("./BRSSD")

train_images = list((dataset_path / "train/images").glob("*.*"))
train_labels = list((dataset_path / "train/labels").glob("*.txt"))
val_images = list((dataset_path / "val/images").glob("*.*"))
val_labels = list((dataset_path / "val/labels").glob("*.txt"))

print(f"Dataset Statistics:")
print(f"  Training: {len(train_images)} images, {len(train_labels)} labels")
print(f"  Validation: {len(val_images)} images, {len(val_labels)} labels")

# Load configuration
with open('brssd_data.yaml', 'r') as f:
    config = yaml.safe_load(f)
    
print(f"\nClasses ({config['nc']}):")
for idx, name in config['names'].items():
    print(f"  {idx}: {name}")

## 4. Visualize Sample Images

In [ ]:
# Display random training images with annotations
import random

def plot_annotated_images(num_samples=6):
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.ravel()
    
    samples = random.sample(train_images, min(num_samples, len(train_images)))
    
    for idx, img_path in enumerate(samples):
        img = cv2.imread(str(img_path))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Read corresponding label
        label_path = dataset_path / "train/labels" / f"{img_path.stem}.txt"
        
        if label_path.exists():
            with open(label_path, 'r') as f:
                annotations = f.readlines()
            
            h, w = img.shape[:2]
            for ann in annotations:
                parts = ann.strip().split()
                if len(parts) >= 5:
                    class_id, x_center, y_center, width, height = map(float, parts[:5])
                    
                    # Convert YOLO format to pixel coordinates
                    x1 = int((x_center - width/2) * w)
                    y1 = int((y_center - height/2) * h)
                    x2 = int((x_center + width/2) * w)
                    y2 = int((y_center + height/2) * h)
                    
                    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    class_name = config['names'].get(int(class_id), f"Class {int(class_id)}")
                    cv2.putText(img, class_name, (x1, y1-10), 
                              cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        
        axes[idx].imshow(img)
        axes[idx].axis('off')
        axes[idx].set_title(img_path.name)
    
    plt.tight_layout()
    plt.show()

plot_annotated_images()

## 5. Initialize YOLOv10 Model

In [ ]:
# Choose model size: n (nano), s (small), m (medium), l (large), x (extra large)
MODEL_SIZE = 'n'  # Start with nano for faster training

model = YOLO(f'yolov10{MODEL_SIZE}.pt')
print(f"✓ YOLOv10-{MODEL_SIZE.upper()} model loaded")

## 6. Train Model

In [ ]:
# Training configuration
results = model.train(
    data='brssd_data.yaml',
    epochs=100,
    imgsz=640,
    batch=16,
    name=f'YOLOv10{MODEL_SIZE}_BRSSD',
    patience=50,
    save=True,
    device=0,  # 0 for GPU, 'cpu' for CPU
    workers=8,
    project='runs/brssd',
    exist_ok=True,
    pretrained=True,
    optimizer='auto',
    verbose=True,
    seed=42,
    val=True,
    plots=True,
    
    # Augmentation
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    degrees=0.0,
    translate=0.1,
    scale=0.5,
    fliplr=0.5,
    mosaic=1.0,
    
    # Learning rate
    lr0=0.01,
    lrf=0.01,
)

## 7. Evaluate Model

In [ ]:
# Validate on test/validation set
metrics = model.val()

print("\nValidation Metrics:")
print(f"  mAP50: {metrics.box.map50:.4f}")
print(f"  mAP50-95: {metrics.box.map:.4f}")
print(f"  Precision: {metrics.box.mp:.4f}")
print(f"  Recall: {metrics.box.mr:.4f}")

## 8. Test Predictions

In [ ]:
# Load best model
best_model = YOLO(f'runs/brssd/YOLOv10{MODEL_SIZE}_BRSSD/weights/best.pt')

# Predict on validation images
test_images = random.sample(val_images, min(6, len(val_images)))

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.ravel()

for idx, img_path in enumerate(test_images):
    results = best_model.predict(str(img_path), conf=0.25)
    
    # Plot results
    annotated = results[0].plot()
    annotated = cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB)
    
    axes[idx].imshow(annotated)
    axes[idx].axis('off')
    axes[idx].set_title(f'Prediction: {img_path.name}')

plt.tight_layout()
plt.show()

## 9. Export Model

In [ ]:
# Export to different formats
best_model.export(format='onnx')  # ONNX format
# best_model.export(format='torchscript')  # TorchScript
# best_model.export(format='tflite')  # TensorFlow Lite

print("✓ Model exported")

## 10. Real-time Inference (Optional)

In [ ]:
# Test on custom image
# Upload your own image and test

# from google.colab import files
# uploaded = files.upload()

# for filename in uploaded.keys():
#     results = best_model.predict(filename, conf=0.25)
#     results[0].show()
#     print(f"Detected {len(results[0].boxes)} traffic signs")